original: https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10  
refer-to: http://blog.suprsonicjetboy.com/entry/2017/04/30/204951

In [ ]:
import os
import re
import sys
import tarfile

In [ ]:
from datetime import datetime
import time

In [ ]:
from six.moves import urllib
import tensorflow as tf
import numpy as np

In [ ]:
IMAGE_SIZE = 24
NUM_CLASSES = 10

## Input Data (Definition)

In [ ]:
def read_jpg(filepath, size=IMAGE_SIZE):
    """Reads image from jpg-file."""
    f = tf.read_file(filepath)
    image = tf.image.decode_jpeg(f, channels=3)
    image = tf.image.resize_images(image, (size, size))
    return image

In [ ]:
def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory."""
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)
    return var

In [ ]:
def _variable_with_stddev(name, shape, stddev, wd=None):
    """Helper to create an initialized Variable with truncated-normal initializer."""
    initializer = tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32)
    var = _variable_on_cpu(name, shape, initializer)
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [ ]:
def inference(images, batch_size=None):
    """Build the CIFAR-10 model."""
    if batch_size is None:
        batch_size = images.get_shape()[0].value
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_stddev('weights', shape=[5, 5, 3, 64], stddev=5e-2)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        # _activation_summary(conv1)

    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_stddev('weights', shape=[5, 5, 64, 64], stddev=5e-2)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        # _activation_summary(conv2)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_stddev('weights', shape=[dim, 384], stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        # _activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_stddev('weights', shape=[384, 192], stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        # _activation_summary(local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_stddev('weights', [192, NUM_CLASSES], stddev=1/192.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        # _activation_summary(softmax_linear)

    return softmax_linear

In [ ]:
def predict(filepath):
    """Predict file with CIFAR-10 model."""
    images = [read_jpg(filepath)]
    summary_op = tf.summary.image('images', images)
    with tf.variable_scope("logits"), tf.name_scope("predict") as scope:
        pred_logits = inference(images, batch_size=1)
        pred_softmax = tf.nn.softmax(pred_logits)
    
    return pred_softmax, summary_op

In [ ]:
filename = './airplane.jpg'

`![./airplane.jpg](./airplane.jpg)`

![./airplane.jpg](./airplane.jpg)  
（Photo by: [フリー素材ぱくたそ](https://www.pakutaso.com/)）

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
def classify(filepath, classes=classes):
    checkpoint_dir = './train'
    checkpoint_path = os.path.join(checkpoint_dir, 'model.ckpt')

    with tf.Graph().as_default():
        logits_op, summary_op = predict(filepath)
        # Build an initialization operation to run below.
        init = tf.global_variables_initializer()
        # Create a loader.
        loader = tf.train.Saver(tf.global_variables())
        
        with tf.Session() as sess:
            sess.run(init)
            ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
            if ckpt and ckpt.model_checkpoint_path:
                # Restores from checkpoint
                loader.restore(sess, ckpt.model_checkpoint_path)
            else:
                print('No checkpoint file found')
                return

            summary_dir = './summary'
            summary_predict_dir = os.path.join(summary_dir, 'predict')
            summary_writer = tf.summary.FileWriter(summary_predict_dir)

            _logits, summary = sess.run([logits_op, summary_op])
            summary_writer.add_summary(summary)

            cls = classes[np.argmax(_logits)]
            print('Prediction:', cls)
            print(_logits)
            
            summary_writer.close()

In [ ]:
classify(filename)

`![./pony.jpg](./pony.jpg)`

![./pony.jpg](./pony.jpg)  
（Photo by: [フリー素材ぱくたそ](https://www.pakutaso.com/)）

In [ ]:
classify('./pony.jpg')

`![./cat.jpg](./cat.jpg)`

![./cat.jpg](./cat.jpg)  
（Photo by: [フリー素材ぱくたそ](https://www.pakutaso.com/)）

In [ ]:
classify('./cat.jpg')